In [4]:
# import tensorflow as tf
# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   raise SystemError('GPU device not found')
# print('Found GPU at: {}'.format(device_name))

# You'll generate plots of attention in order to see which parts of an image
# our model focuses on during captioning
import matplotlib.pyplot as plt

# Scikit-learn includes many helpful utilities
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

import re
import numpy as np  
import os
import time
import json
import random
import pickle
import cv2
import os.path
from os import path
import shutil
from glob import glob
from shutil import copyfile
from pathlib import Path
from matplotlib import pyplot as plt
from IPython.display import clear_output, display
from collections import Counter
import builtins

from sklearn.utils import shuffle

#
# gera imagem com jogadas escritas com fonte manuscrito
#
# -*- coding: utf-8 -*-
import PIL
from PIL import ImageFont
from PIL import Image, ImageDraw
from PIL import ImageColor

from scipy.ndimage.interpolation import map_coordinates
from scipy.ndimage.filters import gaussian_filter

# print(tf.__version__) 

NameError: name 'tf' is not defined

In [5]:
def read_label( path):
  f= open( path)
  d= f.read()
  f.close()
  return d 

def write_label( path, text):
  f= open( path, "w")
  f.write( text)
  f.close()

def show( img, name= 'noname'):
  plt.imshow( img)
  plt.title( name)
  plt.show()

def show_gimage( img, name= 'noname'):
  plt.imshow( img, cmap= 'gray')
  plt.title( name)
  plt.show()

def _load_image( path):
  # return cv2.imread( path, cv2.IMREAD_GRAYSCALE)
  return cv2.imread( path)
  
def write_image( path, img):
  cv2.imwrite( path, img)

def resize( img, scale_percent= 60, verbose=False):
  #scale_percent = 60 # percent of original size
  width = int(img.shape[1] * scale_percent / 100)
  height = int(img.shape[0] * scale_percent / 100)
  dim = (width, height)
  if verbose:
    print( "scale", scale_percent)
    print( "dim", dim)
  # resize image
  return cv2.resize(img, dim, interpolation = cv2.INTER_AREA) 

In [6]:
# https://gist.github.com/erniejunior/601cdf56d2b424757de5
def elastic_transform_2(image, alpha, sigma,random_state=None):
#     print( alpha, sigma)
    """Elastic deformation of images as described in [Simard2003]_.
    .. [Simard2003] Simard, Steinkraus and Platt, "Best Practices for
       Convolutional Neural Networks applied to Visual Document Analysis", in
       Proc. of the International Conference on Document Analysis and
       Recognition, 2003.
    """
    if random_state is None:
        random_state = np.random.RandomState(None)

    shape = image.shape
    dx = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dy = gaussian_filter((random_state.rand(*shape) * 2 - 1), sigma, mode="constant", cval=0) * alpha
    dz = np.zeros_like(dx)

#     x, y, z = np.meshgrid(np.arange(shape[0]), np.arange(shape[1]), np.arange(shape[2]))
    x, y, z = np.meshgrid(np.arange(shape[1]), np.arange(shape[0]), np.arange(shape[2]))
#     print (x.shape)
    indices = np.reshape(y+dy, (-1, 1)), np.reshape(x+dx, (-1, 1)), np.reshape(z, (-1, 1))

    distored_image = map_coordinates(image, indices, order=1, mode='reflect')
    return distored_image.reshape(image.shape)

In [7]:
def transform( img):
    # ELASTIC_ALPHA_SIGMA = ((1201, 10), (1501, 12), (991, 8))
    if type( img)== PIL.Image.Image:
        img= np.asarray( img)
        img= elastic_transform_2(img, 1201, 10)  #random.randint( 991, 1501), random.randint( 8, 12))
        img= PIL.Image.fromarray( img)
        return img
    else:
        return elastic_transform_2(img, 1201, 10)  #random.randint( 991, 1501), random.randint( 8, 12))

In [ ]:
# pgnfile= "C:/mestrado/datasets/handmade/imagens-originais/pgn_30k_pt.txt"
# with open(pgnfile) as f:
#     lines = [line.rstrip() for line in f]
    
# lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
# lines= [ln[8:] for ln in lines]
# lines= [ln.split()[0:32] for ln in lines]

# pgnfile= "C:/mestrado/datasets/handmade/imagens-originais/pgn_30k_eng.txt"
# with open(pgnfile) as f:
#     en_lines = [line.rstrip() for line in f]
    
# en_lines= [re.sub("[\[].*?[\]]", "", ln) for ln in en_lines]
# en_lines= [ln[8:] for ln in en_lines]
# en_lines= [ln.split()[0:32] for ln in en_lines]

# pgnfile= "C:/mestrado/datasets/jogos/pgn_30k_pt.txt"
# with open(pgnfile) as f:
#     lines = [line.rstrip() for line in f]
    
# lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
# lines= [ln[8:] for ln in lines]
# lines= [ln.split() for ln in lines]

# en_pgnfile= "C:/mestrado/datasets/jogos/pgn_30k_eng.txt"
# with open(en_pgnfile) as f:
#     en_lines = [line.rstrip() for line in f]
    
# en_lines= [re.sub("[\[].*?[\]]", "", ln) for ln in en_lines]
# en_lines= [ln[8:] for ln in en_lines]
# en_lines= [ln.split() for ln in en_lines]

In [ ]:
# print( lines[0], en_lines[0])

In [ ]:
# print( len( lines), len(en_lines))

In [ ]:
#
# le todos os fontes
#

In [8]:
new_fonts_files= glob( 'C:/mestrado/datasets/samples_font--2/*.ttf.jpg')

In [9]:
print( len( new_fonts_files))

120


In [10]:
new_fonts_path= [
    os.path.join( 'C:/mestrado/datasets/fonts', 
                 Path( f).name.replace( '.jpg', '')) for f in new_fonts_files]

In [25]:
new_fonts= [ImageFont.truetype(f, 60) for f in new_fonts_path]

In [26]:
fonts= [
    ImageFont.truetype('c:/Windows/Fonts/BRADHITC.TTF', 60),
    ImageFont.truetype('c:/Windows/Fonts/ITCKRIST.TTF', 60),
    ImageFont.truetype('c:/Windows/Fonts/LHANDW.TTF', 60),
    ImageFont.truetype('c:/Windows/Fonts/Inkfree.TTF', 60),
    ImageFont.truetype('c:/Windows/Fonts/OCRAEXT.TTF', 60),
    ImageFont.truetype('c:/Windows/Fonts/TEMPSITC.TTF', 60),
    ImageFont.truetype('c:/Windows/Fonts/PAPYRUS.TTF', 60),
    ImageFont.truetype('c:/Windows/Fonts/MAIAN.TTF', 60)
] 

In [27]:
all_fonts= new_fonts + fonts

In [28]:
print( len( new_fonts), len( fonts), len( all_fonts))

120 8 128


In [13]:
def showbig( img, name= 'noname'):
    plt.figure(figsize = (10,10))    
    plt.imshow( img)
    plt.title( name)
    plt.show()

In [30]:
def sp_noise(npimage,prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(npimage.shape,np.uint8)
    thres = 1 - prob 
    for i in range(npimage.shape[0]):
        for j in range(npimage.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = random.randint( 0, 128)
            else:
                output[i][j] = npimage[i][j]
    return output

def sp_noise_pil( pilimg, prob):
#     print( 'noise', prob)
    img= pilimg
    img= np.asarray( img)
    img= sp_noise(img, prob)
    img= PIL.Image.fromarray( img)
    return img

def distorce( img):
    # ELASTIC_ALPHA_SIGMA = ((1201, 10), (1501, 12), (991, 8))
#     print( 'distorce')
    if type( img)== PIL.Image.Image:
        img= np.asarray( img)
        img= elastic_transform_2(img, 991, 8)  #random.randint( 991, 1501), random.randint( 8, 12))
        img= PIL.Image.fromarray( img)
        return img
    else:
        return elastic_transform_2(img, 991, 8)  #random.randint( 991, 1501), random.randint( 8, 12))
    

# Rotate the foreground
def rotate( pilimg):
#     print( 'rotate')
    foreground= pilimg
    angle_degrees = random.uniform(-2.5,2.5)
#     print( angle_degrees)
    foreground = foreground.rotate(angle_degrees, resample=Image.BICUBIC, expand=True, fillcolor = (255,255,255))

    # Scale the foreground
#     scale = random.random() * .5 + .5 # Pick something between .5 and 1
#     scale = 1
#     new_size = (int(foreground.size[0] * scale), int(foreground.size[1] * scale))
#     foreground = foreground.resize(new_size, resample=Image.BICUBIC)
    return foreground
#     show( foreground)
    

def random_transform( pilimg, p):
    img= pilimg
    if( random.random()< 0.8):
        img= rotate( img)
#     if( random.random()< 0.1):
#         img= distorce( img)
    if( random.random()< 0.25):
        img= sp_noise_pil( img, random.uniform( 0.001, 0.0001))
    
    return img

In [ ]:
# pgn_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v5/*/labels/*.pgn')
# pgn_files= [f for f in pgn_files if 'scan' in f]
# print( len( pgn_files))

# all_pgn= []
# for f in pgn_files:
#     all_pgn.append( read_label( f))
	
# all_pgn= [p.split()[0:16] for p in all_pgn]
# flat_all_pgn = [t for _list in all_pgn for t in _list]

# count16= Counter( flat_all_pgn)

# print( 'len( count16)', len( _count16))  #esperado 462
# counted16= {k: v for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)}
# print( counted16)



In [ ]:
#
# verifica se cobre todos da base randomica
#
files_r= glob( 'C:\mestrado\datasets\carnaval\carnaval_8lines_shuffle_parts\labels\*.pgn')\

all_rand= []
for f in files_r:
    all_rand.append( read_label( f))
    
count16_r= Counter( all_rand)

print( 'len( count16)', len( count16_r))  #esperado 165
counted16_r= {k: v for k, v in sorted(count16_r.items(), key=lambda item: item[1], reverse=True)}
print( counted16_r)

words_r= [item for item in count16_r]

In [14]:
pgn_files_test= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v6/test/labels/*.pgn')
all_pgn= []
for f in pgn_files_test:
    all_pgn.append( read_label( f))
all_pgn= [p.split()[0:16] for p in all_pgn]
flat_all_pgn = [t for _list in all_pgn for t in _list]




len( count16) 165
{'O-O': 115, 'Nf3': 100, 'Nf6': 99, 'd4': 87, 'e4': 76, 'Nc3': 71, 'c5': 63, 'e6': 60, 'Nc6': 59, 'd6': 52, 'd5': 50, 'c4': 43, 'e5': 41, 'g6': 41, 'Be7': 38, 'Nxd4': 37, 'Bg7': 37, 'cxd4': 36, 'a6': 27, 'g3': 26, 'e3': 26, 'Bg2': 24, 'Bd3': 23, 'c6': 22, 'Nbd7': 21, 'd3': 20, 'Bg5': 20, 'Be3': 17, 'c3': 17, 'Bc4': 16, 'Be2': 15, 'Bf4': 14, 'b6': 13, 'Qd2': 12, 'b3': 12, 'Bb7': 12, 'Nbd2': 11, 'exd5': 10, 'Qc2': 9, 'h3': 9, 'f4': 9, 'b5': 9, 'Bb2': 8, 'Nge7': 8, 'Qb6': 8, 'dxe4': 8, 'cxd5': 8, 'Bd6': 8, 'Bb4': 7, 'Nd2': 7, 'Bb5': 7, 'h6': 7, 'Qc7': 6, 'exd4': 6, 'Bxf6': 6, 'f3': 6, 'Be6': 6, 'Nxc6': 6, 'bxc6': 5, 'dxc4': 5, 'Bxc6': 5, 'Qd7': 5, 'Nxe4': 4, 'Qb3': 4, 'Qxd4': 4, 'Qe2': 4, 'Qxd6': 4, 'Nge2': 4, 'Bb5+': 3, 'Bd7': 3, 'Ne4': 3, 'Bg4': 3, 'Bb3': 3, 'Qxc7': 3, 'Nd7': 3, 'Bxc3+': 3, 'bxc3': 3, 'Bxd6': 3, 'Bc5': 3, 'Qa5': 3, 'Nxd5': 3, 'h5': 3, 'Nb5': 3, 'Qa4': 3, 'Bh4': 3, 'Rb8': 3, 'Bg3': 3, 'Nde2': 3, 'Qe7': 3, 'Re1': 2, 'Re8': 2, 'Rb1': 2, 'Ng5': 2, 'Na5': 2

In [58]:
# adiciona os do randomico
flat_all_pgn= flat_all_pgn+ all_rand

count16= Counter( flat_all_pgn)

print( 'len( count16)', len( count16))  #esperado 165
counted16= {k: v for k, v in sorted(count16.items(), key=lambda item: item[1], reverse=True)}
print( counted16)

words= [item for item in count16]
#ordena

len( count16) 175
{'O-O': 218, 'Nf3': 186, 'Nf6': 184, 'd4': 170, 'e4': 140, 'Nc3': 134, 'c5': 121, 'Nc6': 118, 'd5': 113, 'e6': 112, 'd6': 88, 'c4': 80, 'e5': 78, 'Be7': 71, 'g6': 65, 'Bg7': 58, 'a6': 54, 'e3': 49, 'Bg2': 41, 'c6': 41, 'g3': 40, 'Bd3': 40, 'd3': 38, 'Nxd4': 37, 'c3': 37, 'cxd4': 36, 'Nbd7': 34, 'Be3': 33, 'Bg5': 33, 'Bf4': 28, 'Bc4': 28, 'Be2': 28, 'b3': 23, 'b6': 23, 'Nbd2': 22, 'Qd2': 21, 'h3': 19, 'Bb7': 19, 'Bb4': 16, 'Bb2': 16, 'Qb6': 16, 'Qc2': 16, 'Bb5': 16, 'b5': 16, 'Bd6': 16, 'Qc7': 15, 'Bd7': 15, 'Nd2': 13, 'f4': 12, 'Be6': 12, 'Bg4': 11, 'f3': 11, 'a3': 11, 'Nge7': 10, 'exd5': 10, 'h6': 10, 'Bc5': 9, 'Re1': 8, 'dxe4': 8, 'Bb3': 8, 'cxd5': 8, 'Nb6': 8, 'Qb3': 7, 'Qd7': 7, 'O-O-O': 7, 'exd4': 6, 'Nd5': 6, 'Bxf6': 6, 'a4': 6, 'Nge2': 6, 'Nxc6': 6, 'b4': 6, 'Bf5': 6, 'bxc6': 5, 'dxc4': 5, 'Bxc6': 5, 'Qa5': 5, 'Ngf3': 5, 'Nxe4': 4, 'Re8': 4, 'Qxd4': 4, 'Na5': 4, 'Ne4': 4, 'Ba4': 4, 'Qe2': 4, 'Qxd6': 4, 'f5': 4, 'Bd2': 4, 'Nb5': 4, 'Qa4': 4, 'Bf3': 4, 'Nde2': 4,

In [59]:
words.sort()

In [60]:
print( words)

['Ba4', 'Ba5', 'Ba7', 'Bb2', 'Bb3', 'Bb4', 'Bb4+', 'Bb5', 'Bb5+', 'Bb7', 'Bc4', 'Bc5', 'Bd2', 'Bd3', 'Bd6', 'Bd7', 'Be2', 'Be3', 'Be5', 'Be6', 'Be7', 'Bf3', 'Bf4', 'Bf5', 'Bg2', 'Bg3', 'Bg4', 'Bg5', 'Bg7', 'Bh4', 'Bh6', 'Bh7', 'Bxb4', 'Bxc3', 'Bxc3+', 'Bxc4', 'Bxc5', 'Bxc6', 'Bxc7', 'Bxd2+', 'Bxd6', 'Bxd7+', 'Bxf6', 'Kd8', 'Kxd8', 'Na4', 'Na5', 'Na6', 'Nb3', 'Nb4', 'Nb5', 'Nb6', 'Nbd2', 'Nbd7', 'Nbxd2', 'Nc2', 'Nc3', 'Nc5', 'Nc6', 'Nce7', 'Nd2', 'Nd4', 'Nd5', 'Nd6+', 'Nd7', 'Ndb5', 'Nde2', 'Ne1', 'Ne2', 'Ne4', 'Ne5', 'Ne7', 'Nf1', 'Nf3', 'Nf6', 'Nfd7', 'Ng5', 'Nge2', 'Nge7', 'Ngf3', 'Ngf6', 'Nh5', 'Nxc6', 'Nxd4', 'Nxd5', 'Nxd7', 'Nxe4', 'Nxe5', 'Nxf6+', 'O-O', 'O-O-O', 'Qa4', 'Qa5', 'Qa5+', 'Qb3', 'Qb6', 'Qc2', 'Qc3', 'Qc7', 'Qd1', 'Qd2', 'Qd3', 'Qd4', 'Qd5', 'Qd6', 'Qd7', 'Qe1', 'Qe2', 'Qe7', 'Qe8', 'Qf3', 'Qf6', 'Qg6', 'Qh4', 'Qxb2', 'Qxb3', 'Qxc4', 'Qxc7', 'Qxd4', 'Qxd5', 'Qxd6', 'Qxd8+', 'Qxe4+', 'Qxf6', 'Rb1', 'Rb8', 'Rc8', 'Re1', 'Re8', 'a3', 'a4', 'a5', 'a6', 'b3', 'b4', 'b5', '

In [34]:
# pgn_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v7/test/labels/*.pgn')

# all_pgn= []
# for f in pgn_files:
#     all_pgn.append( read_label( f))
	
# lines_test= [p.split()[0:16] for p in all_pgn]

In [ ]:
# lines_test

In [36]:
# flat_test_pgn = [t for _list in lines_test for t in _list]

In [40]:
# [p for p in flat_test_pgn if p not in words]

[]

In [ ]:
# # le os primeiros 16
# pgnfile= "C:/mestrado/datasets/jogos/pgn_30k_eng.txt"
# with open(pgnfile) as f:
# 	lines = [line.rstrip() for line in f]
# lines= [re.sub("[\[].*?[\]]", "", ln) for ln in lines]
# lines= [ln[8:] for ln in lines]
# lines16= [ln.split()[:16] for ln in lines]
# flat_all_pgn = [t for _list in lines16 for t in _list]
# count16= Counter( flat_all_pgn)
# words= [item for item in count16]


In [15]:
# 
# transforma de portugues para ingles, para gerar pgn
def pt_to_pgn( w):
    # R=> K
    # T=> R
    # C=> N
    # B=> B
    # D=> Q

#     print( w, '=> ')
    png= w.replace( 'R', 'K').replace( 'T', 'R').replace( 'C', 'N').replace( 'D', 'Q')
#     print( png)
    return png

def to_pgn( label):
    return [pt_to_pgn( w) for w in label]

In [16]:
def pgn_to_pt( w):
    # R => T
    # K => R
    # N => C
    # Q => D
    
    return w.replace( 'R', 'T').replace( 'K', 'R').replace( 'N', 'C').replace( 'Q', 'D')

In [17]:
def random_label_from_vocab():
    label= []
    for i in range( 0, 16):
        label.append( 
            pgn_to_pt( words[random.randrange( 0, len( words))])
        )
    return label


print( random_label_from_vocab())

['Be5', 'Bg3', 'e6', 'd4', 'g6', 'Cge7', 'Db6', 'Dd4', 'Cf6', 'Bxb4', 'Cfd7', 'Cxc6', 'Bg5', 'Bg5', 'Bg5', 'De8']


In [19]:
dd= {w:0 for w in words}

In [20]:
for i in range( 0, 10000*16):
    w= words[random.randrange( 0, len( words))]
    dd[ w]= dd[ w]+1
    

In [21]:
print( dd)

{'e4': 1016, 'c5': 938, 'Nf3': 978, 'e6': 973, 'd4': 946, 'cxd4': 961, 'Nxd4': 916, 'a6': 938, 'c4': 959, 'Qc7': 985, 'Nc3': 935, 'Nf6': 926, 'Be3': 977, 'Bb4': 946, 'Qd2': 955, 'Nxe4': 1013, 'd6': 923, 'Bb5+': 907, 'Bd7': 967, 'Bxd7+': 1023, 'Nxd7': 1016, 'O-O': 959, 'Ngf6': 972, 'Re1': 976, 'e5': 966, 'c3': 949, 'Be7': 1023, 'd3': 972, 'g3': 894, 'd5': 946, 'b3': 967, 'Bb2': 939, 'Bg2': 969, 'Nbd7': 1043, 'c6': 967, 'Re8': 949, 'Nc6': 978, 'g6': 1046, 'Bg7': 916, 'Nge7': 1022, 'Rb1': 984, 'Bf4': 966, 'e3': 921, 'Qb6': 991, 'Qb3': 952, 'Qc2': 952, 'Nd2': 921, 'Bc4': 952, 'h3': 946, 'exd4': 1012, 'Qxd4': 986, 'Ng5': 941, 'exd5': 998, 'Na5': 940, 'dxc6': 978, 'bxc6': 984, 'Bd3': 954, 'Nd5': 936, 'f4': 1013, 'Ne4': 946, 'dxe4': 973, 'Bg4': 976, 'Nbd2': 962, 'Qxb2': 920, 'Bb5': 1008, 'Ba4': 953, 'b5': 1046, 'Bb3': 976, 'cxd5': 956, 'Bg5': 966, 'Bxf6': 925, 'Be2': 1023, 'b6': 981, 'Bb7': 959, 'dxc4': 1053, 'a4': 1067, 'cxb3': 955, 'Qxb3': 982, 'Qd3': 967, 'Qc3': 959, 'Bxc7': 961, 'Qxc7': 1

In [22]:
min( [v for (k,v) in dd.items()])

894

In [23]:
max( [v for (k,v) in dd.items()])

1067

In [62]:
img = Image.open("C:/mestrado/datasets/syn/template/template-2x.jpg")  # load base image
dctx = ImageDraw.Draw(img)  # create drawing context
dh_top= 14
dh_bottom= 26
h= (img.height - dh_top - dh_bottom)/25.0
w= img.width/4.0

margin_left= w/4
margin_top= h/6

# def resize_fixed( img):
#     _dd = (271, 230) #width, height
#     return cv2.resize(img, _dd, interpolation = cv2.INTER_AREA) 

def generate_for( line):
#     if fnt is None:
#         fnt= random.choice( fonts)
#     print( fnt.getname())
    
    img = Image.open("C:/mestrado/datasets/syn/template/template-2x.jpg")  # load base image
    dctx = ImageDraw.Draw(img)  # create drawing context

    i= 0
    for ln in range( 0, 8):
        for col in range( 0, 2):
            if i>= len( line):
                del dctx
                return img
        
            fnt= random.choice( all_fonts)
            txtsz = dctx.textsize(line[i], fnt)
        
            x= w* col + (w - txtsz[0])/2
            if random.random()< 0.7:
                xmargem= int( (w - txtsz[0])/3)
                x+= random.randint( -xmargem, xmargem)
            y= dh_top + h*ln + (h - txtsz[1])/2 #+ margin_top

            dctx.text( (x, y), line[i], font=fnt, fill="#000000")
            i= i+1
            
    del dctx 
    return img

def cut( img):
    h= 600
    w= 690 #345

    img1= img[:h, :w]
    return img1

def generate_one( label): #pt_line, pgn, no, folder):
#     id_font= no%len( all_fonts)
    img= generate_for( label)
    img= cut( np.asarray(img))
    img= PIL.Image.fromarray(np.uint8(img))
    return img


# def __generate():
#     _files= label_files
# #     print( 'Gerando ', len( _files))
#     for i in range( 0, len( _files)):
#         if i%100== 0:
#             print( i)
#         img_file= _files[i].replace( 'labels', 'images.new').replace( '.pgn', '.jpg') 
#         img= generate_one( i, labelof( _files[i])[0:16], img_file)
        
#         img= random_transform( img, 0.3)
#         img.save( img_file)
# #         print( img_file)
# #         show( img__)
# #         show( transform( img__))
# #         return img
    
# __generate()
# img1= __generate()
# show( img1)
# pilimg= img1
# npimg= np.asarray( img1)


def __generate_random( N):
    dataset_dir= 'C:/mestrado/datasets/unified/random-32k-8lines--v9'
    
    N_train= int(N*0.8)
    print( 'N', N, 'N_train', N_train)
    
    for i in range( 0, N):
        if( i%100== 0):
            clear_output(wait=True)
            print( i, '....')
        label_pt= random_label_from_vocab()
        img= generate_one( label_pt)
        img= random_transform( img, 0.3)
        pgn= to_pgn( label_pt)

        folder= 'train' if i< N_train else 'valid'
        
        # save
#         show( img)
        img_file= os.path.join( dataset_dir, folder, 'images', 'random-syn-'+ str(i+1).zfill( 5)+'.jpg')
        img.save( img_file)
        write_label( img_file.replace( 'images', 'labels').replace( '.jpg', '.pgn'), 
                    " ".join( to_pgn( label_pt)))

__generate_random( 20000)
print( 'DONE!')

19900 ....
DONE!


In [33]:
# def gererate_all_labels():
#     pgn_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v7/*/labels/*.pgn')
#     all_pgn= []
#     for f in pgn_files:
#         all_pgn.append( read_label( f))
        
#     all_pgn= [' '.join( p.split()[0:16]) for p in all_pgn]
    
#     write_label( 'C:/mestrado/datasets/unified/random-32k-8lines--v7/all-labels-.pgn', 
#            '\n'.join( all_pgn))
    
# gererate_all_labels()

In [35]:
words.sort()

In [61]:
#somente 1 linha com as words ordenado
write_label( 'C:/mestrado/datasets/unified/random-32k-8lines--v9/all-labels-.pgn', 
           ' '.join( words))

In [ ]:
# pgn_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v6/*/labels/*.pgn')

In [ ]:
# all_pgn= []
# for f in pgn_files:
#     all_pgn.append( read_label( f))

# pat= "Nc3 d5 c4"

In [ ]:
# for i in range( 0, len( all_pgn)):
#     if all_pgn[i].startswith( "Nc3 d5"):
#         print( i, all_pgn[i])

In [41]:
# 
# copia test set imagem original
#
img_files= glob( 'C:/mestrado/datasets/unified/random-32k-8lines--v9/test/images/*.jpg')

In [42]:
len( img_files)

114

In [50]:
origin_dir= "C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images"
for f in img_files:
    f_from= os.path.join( origin_dir, Path( f).name.replace('fullpage','line_1_8'))
    print( f_from)
    copyfile( f_from, f)
    

C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0004_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0015_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0023_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0029_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0032_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0033_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0038_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0040_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0042_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0051_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8

C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0544_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0551_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0553_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0554_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0565_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0569_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0578_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0580_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0589_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8lines/carnaval_8lines/images\carnaval_0590_line_1_8.jpg
C:/mestrado/datasets/carnaval/carnaval_8

In [53]:
len( files)

1426

len( count16) 102
{'O-O': 103, 'Nf3': 86, 'Nf6': 85, 'd4': 83, 'e4': 64, 'Nc3': 63, 'd5': 63, 'Nc6': 59, 'c5': 58, 'e6': 52, 'c4': 37, 'e5': 37, 'd6': 36, 'Be7': 33, 'a6': 27, 'g6': 24, 'e3': 23, 'Bg7': 21, 'c3': 20, 'c6': 19, 'd3': 18, 'Bg2': 17, 'Bd3': 17, 'Be3': 16, 'Bf4': 14, 'g3': 14, 'Nbd7': 13, 'Be2': 13, 'Bg5': 13, 'Bd7': 12, 'Bc4': 12, 'b3': 11, 'Nbd2': 11, 'a3': 10, 'h3': 10, 'b6': 10, 'Qc7': 9, 'Bb5': 9, 'Qd2': 9, 'Bb4': 9, 'Bg4': 8, 'Bb2': 8, 'Qb6': 8, 'Bd6': 8, 'Qc2': 7, 'Bb7': 7, 'b5': 7, 'Re1': 6, 'Nd2': 6, 'Bc5': 6, 'Be6': 6, 'Nb6': 6, 'Bb3': 5, 'f3': 5, 'b4': 5, 'Bf5': 5, 'O-O-O': 5, 'a4': 4, 'Nd5': 4, 'Qb3': 3, 'Ngf3': 3, 'h6': 3, 'a5': 3, 'f4': 3, 'Ngf6': 2, 'Re8': 2, 'Ne5': 2, 'Na5': 2, 'Ba4': 2, 'f5': 2, 'Nb3': 2, 'Qa5': 2, 'Nge7': 2, 'Nfd7': 2, 'Bf3': 2, 'Nb4': 2, 'Ba7': 2, 'Ne1': 2, 'Nge2': 2, 'Qd7': 2, 'Bd2': 2, 'Nd4': 1, 'Nh5': 1, 'Ng5': 1, 'Qe7': 1, 'Ne7': 1, 'Na6': 1, 'Ne2': 1, 'Ba5': 1, 'Rb1': 1, 'Nc5': 1, 'Ne4': 1, 'Qa4': 1, 'Qf3': 1, 'Bh7': 1, 'Na4': 1, 'h

In [57]:
for w in words_r:
    if w not in words:
        print( w)

Nd4
Ne5
Nh5
Nb4
Ba7
Nc5
Bh7
Na4
h4
Rc8
